In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import os

In [3]:
#load data
df = pd.read_csv("/home/julian/Project/code/quora_duplicate_questions.tsv",delimiter='\t')

names=df.columns.values
print(names)
print(df.shape)

['id' 'qid1' 'qid2' 'question1' 'question2' 'is_duplicate']
(404290, 6)


In [4]:
#training for embedding
import gensim


In [5]:
df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))


In [6]:

questions=list(df['question1'])+list(df['question2'])
c=0
for i in tqdm(questions):
    questions[c]=list(gensim.utils.tokenize(i,deacc=True,lower=True))
    c+=1

100%|██████████| 808580/808580 [08:56<00:00, 1508.09it/s]


In [7]:
from gensim.models import Word2Vec
model=Word2Vec(questions,size=10,min_count=5)

In [8]:
print(model)

Word2Vec(vocab=29165, size=10, alpha=0.025)


In [9]:
import tensorflow as tf

In [13]:
x1=df['question1']
x2=df['question2']
y=df['is_duplicate']

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence,text
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,Merge

Using TensorFlow backend.


In [ ]:
tk=text.Tokenizer(num_words=200000)
max_len=40
tk.fit_on_texts(list(df.question1.values)+list(df.question2.values))
x1 = tk.texts_to_sequences(df.question1.values)
x1 = sequence.pad_sequences(x1,maxlen = max_len)

x2 = tk.texts_to_sequences(df.question2.values)
x2 = sequence.pad_sequences(x1,maxlen = max_len)

In [ ]:
nb_words=len(word_index)+1

In [12]:
m=df.shape[0]
train_start = 0
train_end = int(.5*m)
test_start = int(.5*m)
test_end = m

x1_train=x1[train_start:train_end]
x2_train=x2[train_start:train_end]
y_train=y[train_start:train_end]
x1_test=x1[test_start:test_end]
x2_test=x2[test_start:test_end]
y_test=y[test_start:test_end]

In [ ]:
print(x1_train.shape)
print(x1_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
embedding_matrix = np.zeros((nb_words, 10))

for word, i in word_index.items():
    embedding_vector = None 
    if word in model.wv.vocab:
        embedding_vector = model[word]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_vecor_length = 10
model1 = Sequential()
model1.add(Embedding(nb_words, embedding_vecor_length,weights=[embedding_matrix], input_length=max_len))
x1=model1.add(LSTM(100,return_sequences=True))
#model1.add(Dense(1, activation='sigmoid'))
#model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

In [ ]:
embedding_vecor_length = 10
model2 = Sequential()
model2.add(Embedding(nb_words, embedding_vecor_length,weights=[embedding_matrix], input_length=max_len))
model2.add(LSTM(100,return_sequences=True))
#model2.add(Dense(1, activation='sigmoid'))
#model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())